In [ ]:
from cellpose import models



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.8.0+cu126! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




In [ ]:
# import napari
from pathlib import Path
# from skimage import io
import os, glob
import czifile
import numpy as np
from tqdm.auto import tqdm
import cellpose

In [7]:

# Path to your files
files_dir = Path(r'Z:\Zeiss\Rebecca\zeisslightsheet7\20250814')

# Use string when passing to glob
files = glob.glob(str(files_dir / '*.czi'))

# Get file sizes in bytes
file_sizes = [os.path.getsize(f) for f in files]

# Convert sizes
file_sizes_mb = [s / (1024**2) for s in file_sizes]
total_size_gb = sum(file_sizes) / (1024**3)

# Print summary
for f, s in zip(files, file_sizes_mb):
    print(f"{Path(f).name}: {s:.2f} MB")

print(f"\nNumber of files: {len(files)}")
print(f"Total stacked size: {total_size_gb:.2f} GB")

20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(1).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(10).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(100).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(101).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(102).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(103).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(104).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(105).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(106).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(107).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(108).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(109).czi: 5653.6

In [8]:
test_fn = Path(files_dir, '20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(152).czi')

In [9]:
%%time

image = np.squeeze(czifile.imread(test_fn))

CPU times: total: 11.3 s
Wall time: 18.1 s


In [10]:
image.shape

(2, 402, 1920, 1920)

In [15]:
from cellpose import models


ImportError: DLL load failed while importing _imaging: The operating system cannot run %1.

In [ ]:
model = models.Cellpose(model_type="cyto2", gpu=True)

In [ ]:

# --- robust per-channel normalization to [0,1] ---
img = image.astype(np.float32)
for c in range(img.shape[0]):
    p1, p99 = np.percentile(img[c], (1, 99))
    img[c] = np.clip((img[c] - p1) / max(p99 - p1, 1e-6), 0, 1)

# --- pseudo max-projection across channels for each Z ---
# result: (Z, Y, X)
combo = np.max(img, axis=0)

# --- Cellpose 2D over Z ---

masks = []
for z in tqdm(range(combo.shape[0]), desc="Cellpose on pseudo-max Z"):
    mask, _, _, _ = model.eval(combo[z], channels=[0, 0], diameter=None)
    masks.append(mask)
masks = np.stack(masks)  # (Z, Y, X)


In [ ]:

# --- visualize in napari ---
viewer = napari.Viewer(title="Pseudo-max segmentation")
viewer.add_image(combo, name="combo(ZYX)")
viewer.add_labels(masks, name="masks")
print("combo:", combo.shape, "masks:", masks.shape)